In [18]:
import numpy as np
import pandas as pd
import json
import ast
from tqdm import tqdm

In [19]:
pwd

'/Users/mingyuanma/Desktop/HAI/util'

### Only keep those reasonable data (dropna)

In [36]:
df2 = pd.read_csv("raw-intervention-final-4.csv")
df2 = df2.loc[:,["ResponseId", "round_1_actions", "round_2_actions", "round_3_actions", "round_4_actions", "round_5_actions", "round_6_actions"]]
df2 = df2.dropna().reset_index().iloc[:,1:]

In [46]:
# df = pd.read_csv("new.csv")
df = pd.read_csv("pilot50sa.csv")
# df = df.loc[:,["ResponseId", "intervention", "round_1_actions", "round_2_actions", "round_3_actions", "round_4_actions", "round_5_actions", "round_6_actions"]]
df = df.loc[:,["ResponseId", "round_1_actions", "round_2_actions", "round_3_actions", "round_4_actions", "round_5_actions", "round_6_actions"]]
df = df.dropna().reset_index().iloc[:,1:]

In [21]:
dat = pd.read_csv("pilot50sa.csv")
# dat = dat.loc[:,["ResponseId", "intervention", "round_1_states", "round_2_states", "round_3_states", "round_4_states", "round_5_states", "round_6_states"]]
dat = dat.loc[:,["ResponseId", "round_1_states", "round_2_states", "round_3_states", "round_4_states", "round_5_states", "round_6_states"]]
dat = dat.dropna().reset_index().iloc[:,1:]

### Start on state

In [24]:
zzx = pd.DataFrame(json.loads(dat["round_1_states"][1]))
zzx

JSONDecodeError: Expecting ':' delimiter: line 1 column 5 (char 4)

In [ ]:


# 0~chef, 1~sou_chef, 2~server
# 99 ~ available, 100 ~ finished
final12 = pd.DataFrame()
for round in ["round_1_states", "round_2_states"]:
  out = pd.DataFrame()
  ID = dat["ResponseId"]
  for kim in tqdm(np.arange(len(ID))):
    zzx = pd.DataFrame(json.loads(dat[round][kim]))
    chef_order = [state[0]["order_index"] for state in zzx["worker_states"]]
    chef_task = [state[0]["task"] for state in zzx["worker_states"]]
    chef_left_ticks = [state[0]["ticks"] for state in zzx["worker_states"]]

    sou_chef_order = [state[1]["order_index"] for state in zzx["worker_states"]]
    sou_chef_task = [state[1]["task"] for state in zzx["worker_states"]]
    sou_chef_left_ticks = [state[1]["ticks"] for state in zzx["worker_states"]]

    server_order = [state[2]["order_index"] for state in zzx["worker_states"]]
    server_task = [state[2]["task"] for state in zzx["worker_states"]]
    server_left_ticks = [state[2]["ticks"] for state in zzx["worker_states"]]

    ticks = np.arange(len(chef_order))

    # t01, order 0, task 1, 12 in total 
    t00 = ["available"] * len(ticks)
    t01 = ["unavailable"] * len(ticks)
    t02 = ["unavailable"] * len(ticks)

    t10 = ["available"] * len(ticks)
    t11 = ["unavailable"] * len(ticks)
    t12 = ["unavailable"] * len(ticks)

    t20 = ["available"] * len(ticks)
    t21 = ["unavailable"] * len(ticks)
    t22 = ["unavailable"] * len(ticks)

    t30 = ["available"] * len(ticks)
    t31 = ["unavailable"] * len(ticks)
    t32 = ["unavailable"] * len(ticks)

    matrix = [[t00, t01, t02], [t10, t11, t12], [t20, t21, t22], [t30, t31, t32]]

    i = 0
    for i in np.arange(len(ticks)):
      order_chef = chef_order[i]
      task_chef = chef_task[i]
      if order_chef != None and task_chef != None:
        matrix[order_chef][task_chef][i] = "chef"

        for jisoo in np.arange(2):
          if task_chef == jisoo and chef_left_ticks[i] == 1:
            new1 = matrix[order_chef][jisoo+1][:i+1]
            new2 = ["available"]*(len(ticks) - i - 1)
            matrix[order_chef][jisoo+1] = new1 + new2

        for mmy in np.arange(3):
          if task_chef == mmy and chef_left_ticks[i] == 1:
            new1 = matrix[order_chef][mmy][:i+1]
            new2 = ["finished"]*(len(ticks) - i - 1)
            matrix[order_chef][mmy] = new1 + new2

      order_sou_chef = sou_chef_order[i]
      task_sou_chef = sou_chef_task[i]
      if order_sou_chef != None and task_sou_chef != None:
        matrix[order_sou_chef][task_sou_chef][i] = "sou_chef"

        for jisoo in np.arange(2):
          if task_sou_chef == jisoo and sou_chef_left_ticks[i] == 1:
            new1 = matrix[order_sou_chef][jisoo+1][:i+1]
            new2 = ["available"]*(len(ticks) - i - 1)
            matrix[order_sou_chef][jisoo+1] = new1 + new2


        for mmy in np.arange(3):
          if task_sou_chef == mmy and sou_chef_left_ticks[i] == 1:
            new1 = matrix[order_sou_chef][mmy][:i+1]
            new2 = ["finished"]*(len(ticks) - i - 1)
            matrix[order_sou_chef][mmy] = new1 + new2


      order_server = server_order[i]
      task_server = server_task[i]
      if order_server != None and task_server != None:
        matrix[order_server][task_server][i] = "server"

        for jisoo in np.arange(2):
          if task_server == jisoo and server_left_ticks[i] == 1:
            new1 = matrix[order_server][jisoo+1][:i+1]
            new2 = ["available"]*(len(ticks) - i - 1)
            matrix[order_server][jisoo+1] = new1 + new2

        for mmy in np.arange(3):
          if task_server == mmy and server_left_ticks[i] == 1:
            new1 = matrix[order_server][mmy][:i+1]
            new2 = ["finished"]*(len(ticks) - i - 1)
            matrix[order_server][mmy] = new1 + new2


    player = pd.DataFrame()
    player["ID"] = [ID[kim]] * len(ticks)
    player["round"] = [round] * len(ticks)
    player["chef_order"] = chef_order
    player["chef_task"] = chef_task
    player["chef_left_ticks"] = chef_left_ticks
    player["sou_chef_order"] = sou_chef_order
    player["sou_chef_task"] = sou_chef_task
    player["sou_chef_left_ticks"] = sou_chef_left_ticks
    player["server_order"] = server_order
    player["server_task"] = server_task
    player["server_left_ticks"] = server_left_ticks
    player["ticks"] = ticks
    player["order0_task0"] = matrix[0][0]
    player["order0_task1"] = matrix[0][1]
    player["order0_task2"] = matrix[0][2]
    player["order1_task0"] = matrix[1][0]
    player["order1_task1"] = matrix[1][1]
    player["order1_task2"] = matrix[1][2]
    player["order2_task0"] = matrix[2][0]
    player["order2_task1"] = matrix[2][1]
    player["order2_task2"] = matrix[2][2]
    player["order3_task0"] = matrix[3][0]
    player["order3_task1"] = matrix[3][1]
    player["order3_task2"] = matrix[3][2]

    out = pd.concat([out, player])
  final12 = pd.concat([final12, out])

final12

,ID,round,chef_order,chef_task,chef_left_ticks,sou_chef_order,sou_chef_task,sou_chef_left_ticks,server_order,server_task,...,order0_task2,order1_task0,order1_task1,order1_task2,order2_task0,order2_task1,order2_task2,order3_task0,order3_task1,order3_task2
0,R_ZIZguiYOWRX744x,round_1_states,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,unavailable,available,unavailable,unavailable,available,unavailable,unavailable,available,unavailable,unavailable
1,R_ZIZguiYOWRX744x,round_1_states,0.0,0.0,1.0,1.0,0.0,2.0,2.0,0.0,...,unavailable,sou_chef,unavailable,unavailable,server,unavailable,unavailable,available,unavailable,unavailable
2,R_ZIZguiYOWRX744x,round_1_states,NaN,NaN,NaN,1.0,0.0,1.0,2.0,0.0,...,unavailable,sou_chef,unavailable,unavailable,server,unavailable,unavailable,available,unavailable,unavailable
3,R_ZIZguiYOWRX744x,round_1_states,0.0,1.0,4.0,NaN,NaN,NaN,2.0,0.0,...,unavailable,finished,available,unavailable,server,unavailable,unavailable,available,unavailable,unavailable
4,R_ZIZguiYOWRX744x,round_1_states,0.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,...,unavailable,finished,available,unavailable,finished,available,unavailable,available,unavailable,unavailable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,R_oXk1uY70EL4SaM9,round_2_states,3.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,...,available,finished,finished,finished,finished,finished,finished,finished,chef,unavailable
16,R_oXk1uY70EL4SaM9,round_2_states,3.0,1.0,3.0,NaN,NaN,NaN,0.0,2.0,...,server,finished,finished,finished,finished,finished,finished,finished,chef,unavailable
17,R_oXk1uY70EL4SaM9,round_2_states,3.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,...,finished,finished,finished,finished,finished,finished,finished,finished,chef,unavailable
18,R_oXk1uY70EL4SaM9,round_2_states,3.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,...,finished,finished,finished,finished,finished,finished,finished,finished,chef,unavailable


In [ ]:
pd.DataFrame(json.loads(dat["round_3_states"][2]))["worker_states"][0]

[{'order_index': None, 'task': None, 'ticks': None},
 {'order_index': None, 'task': None, 'ticks': None}]

In [ ]:


# 0~chef, 1~sou_chef, 2~server
# 99 ~ available, 100 ~ finished
final36 = pd.DataFrame()
for round in ["round_3_states", "round_4_states", "round_5_states", "round_6_states"]:
  out = pd.DataFrame()
  ID = dat["ResponseId"]
  for kim in tqdm(np.arange(len(ID))):
    zzx = pd.DataFrame(json.loads(dat[round][kim]))

    sou_chef_order = [state[0]["order_index"] for state in zzx["worker_states"]]
    sou_chef_task = [state[0]["task"] for state in zzx["worker_states"]]
    sou_chef_left_ticks = [state[0]["ticks"] for state in zzx["worker_states"]]

    server_order = [state[1]["order_index"] for state in zzx["worker_states"]]
    server_task = [state[1]["task"] for state in zzx["worker_states"]]
    server_left_ticks = [state[1]["ticks"] for state in zzx["worker_states"]]

    ticks = np.arange(len(sou_chef_order))

    # t01, order 0, task 1, 12 in total 
    t00 = ["available"] * len(ticks)
    t01 = ["unavailable"] * len(ticks)
    t02 = ["unavailable"] * len(ticks)

    t10 = ["available"] * len(ticks)
    t11 = ["unavailable"] * len(ticks)
    t12 = ["unavailable"] * len(ticks)

    t20 = ["available"] * len(ticks)
    t21 = ["unavailable"] * len(ticks)
    t22 = ["unavailable"] * len(ticks)

    t30 = ["available"] * len(ticks)
    t31 = ["unavailable"] * len(ticks)
    t32 = ["unavailable"] * len(ticks)

    matrix = [[t00, t01, t02], [t10, t11, t12], [t20, t21, t22], [t30, t31, t32]]

    i = 0
    for i in np.arange(len(ticks)):

      order_sou_chef = sou_chef_order[i]
      task_sou_chef = sou_chef_task[i]
      if order_sou_chef != None and task_sou_chef != None:
        matrix[order_sou_chef][task_sou_chef][i] = "sou_chef"

        for jisoo in np.arange(2):
          if task_sou_chef == jisoo and sou_chef_left_ticks[i] == 1:
            new1 = matrix[order_sou_chef][jisoo+1][:i+1]
            new2 = ["available"]*(len(ticks) - i - 1)
            matrix[order_sou_chef][jisoo+1] = new1 + new2


        for mmy in np.arange(3):
          if task_sou_chef == mmy and sou_chef_left_ticks[i] == 1:
            new1 = matrix[order_sou_chef][mmy][:i+1]
            new2 = ["finished"]*(len(ticks) - i - 1)
            matrix[order_sou_chef][mmy] = new1 + new2


      order_server = server_order[i]
      task_server = server_task[i]
      if order_server != None and task_server != None:
        matrix[order_server][task_server][i] = "server"

        for jisoo in np.arange(2):
          if task_server == jisoo and server_left_ticks[i] == 1:
            new1 = matrix[order_server][jisoo+1][:i+1]
            new2 = ["available"]*(len(ticks) - i - 1)
            matrix[order_server][jisoo+1] = new1 + new2

        for mmy in np.arange(3):
          if task_server == mmy and server_left_ticks[i] == 1:
            new1 = matrix[order_server][mmy][:i+1]
            new2 = ["finished"]*(len(ticks) - i - 1)
            matrix[order_server][mmy] = new1 + new2


    player = pd.DataFrame()
    player["ID"] = [ID[kim]] * len(ticks)
    player["round"] = [round] * len(ticks)
    player["sou_chef_order"] = sou_chef_order
    player["sou_chef_task"] = sou_chef_task
    player["sou_chef_left_ticks"] = sou_chef_left_ticks
    player["server_order"] = server_order
    player["server_task"] = server_task
    player["server_left_ticks"] = server_left_ticks
    player["ticks"] = ticks
    player["order0_task0"] = matrix[0][0]
    player["order0_task1"] = matrix[0][1]
    player["order0_task2"] = matrix[0][2]
    player["order1_task0"] = matrix[1][0]
    player["order1_task1"] = matrix[1][1]
    player["order1_task2"] = matrix[1][2]
    player["order2_task0"] = matrix[2][0]
    player["order2_task1"] = matrix[2][1]
    player["order2_task2"] = matrix[2][2]
    player["order3_task0"] = matrix[3][0]
    player["order3_task1"] = matrix[3][1]
    player["order3_task2"] = matrix[3][2]

    out = pd.concat([out, player])
  final36 = pd.concat([final36, out])

final36

,ID,round,sou_chef_order,sou_chef_task,sou_chef_left_ticks,server_order,server_task,server_left_ticks,ticks,order0_task0,...,order0_task2,order1_task0,order1_task1,order1_task2,order2_task0,order2_task1,order2_task2,order3_task0,order3_task1,order3_task2
0,R_ZIZguiYOWRX744x,round_3_states,NaN,NaN,NaN,NaN,NaN,NaN,0,available,...,unavailable,available,unavailable,unavailable,available,unavailable,unavailable,available,unavailable,unavailable
1,R_ZIZguiYOWRX744x,round_3_states,0.0,0.0,2.0,1.0,0.0,3.0,1,sou_chef,...,unavailable,server,unavailable,unavailable,available,unavailable,unavailable,available,unavailable,unavailable
2,R_ZIZguiYOWRX744x,round_3_states,0.0,0.0,1.0,1.0,0.0,2.0,2,sou_chef,...,unavailable,server,unavailable,unavailable,available,unavailable,unavailable,available,unavailable,unavailable
3,R_ZIZguiYOWRX744x,round_3_states,NaN,NaN,NaN,1.0,0.0,1.0,3,finished,...,unavailable,server,unavailable,unavailable,available,unavailable,unavailable,available,unavailable,unavailable
4,R_ZIZguiYOWRX744x,round_3_states,2.0,0.0,2.0,NaN,NaN,NaN,4,finished,...,unavailable,finished,available,unavailable,sou_chef,unavailable,unavailable,available,unavailable,unavailable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,R_oXk1uY70EL4SaM9,round_6_states,3.0,1.0,2.0,2.0,1.0,4.0,28,finished,...,finished,finished,finished,finished,finished,server,unavailable,finished,sou_chef,unavailable
29,R_oXk1uY70EL4SaM9,round_6_states,3.0,1.0,1.0,2.0,1.0,3.0,29,finished,...,finished,finished,finished,finished,finished,server,unavailable,finished,sou_chef,unavailable
30,R_oXk1uY70EL4SaM9,round_6_states,NaN,NaN,NaN,2.0,1.0,2.0,30,finished,...,finished,finished,finished,finished,finished,server,unavailable,finished,finished,available
31,R_oXk1uY70EL4SaM9,round_6_states,3.0,2.0,2.0,2.0,1.0,1.0,31,finished,...,finished,finished,finished,finished,finished,server,unavailable,finished,finished,sou_chef


In [ ]:
final12.to_csv("state_12.csv")

In [ ]:
final36.to_csv("state_36.csv")

### Start on action

for round 1-2

In [47]:
# ticks is array; json.loads(...)
def player_round12(ticks):
  tick_array = []
  time = 1
  worker_this_round = []
  order_this_round = []
  task_this_round = []
  action_this_tick = []
  for tick in ticks:
    inn = False
    worker_this_tick = {0:1, 1:1, 2:2} #key is worker
    order_this_tick = {0:6, 1:6, 2:6} #key is worker
    task_this_tick = {0:6, 1:6, 2:6} #key is worker
    for action in tick:
      if list(action.values()) not in action_this_tick:
        #print(time, "action: ", action, "values: ", list(action.values()), "list: ", action_this_tick)
        worker = action["worker_index"]
        worker_this_tick[worker] = worker
        order_this_tick[worker] = action["order_index"]
        task_this_tick[worker] = action["task"]
        action_this_tick.append(list(action.values()))
      # else:
        #print(time, "action: ", action, "values: ", list(action.values()), "list: ", action_this_tick)
    time_this_tick = [time for _ in np.arange(3)]
    tick_array.extend(time_this_tick)
    time += 1
    worker_this_round.extend(list(worker_this_tick.values()))
    order_this_round.extend(list(order_this_tick.values()))
    task_this_round.extend(list(task_this_tick.values()))
  tick_array, worker_this_round, order_this_round, task_this_round = np.array(tick_array), np.array(worker_this_round), np.array(order_this_round), np.array(task_this_round)
  out = pd.DataFrame()
  out["tick"], out["worker"], out["order"], out["task"] = tick_array, worker_this_round, order_this_round, task_this_round
  return out


# jm = player_round12(json.loads(df[df.ResponseId == "R_12g85TXz0g5zQSv"]["round_2_actions"][119]))
# jm[jm.order == 6]
# jm

action_df_12 = pd.DataFrame()
for id in tqdm(np.arange(len(df["ResponseId"]))):
  for round in ["round_1_actions", "round_2_actions"]:
    df_round = player_round12(json.loads(df[round][id]))
    length = len(df_round.iloc[:,1])
    ResponseId = np.array([df.iloc[id, 0] for _ in np.arange(length)])
    intervention = np.array([df.iloc[id, 1] for _ in np.arange(length)])
    round_array = np.array([round.split("_")[1] for _ in np.arange(length)])
    df_round["ResponseId"] = ResponseId
    df_round["intervention"] = intervention
    df_round["round"] = round_array
    df_round = df_round[["ResponseId", "intervention", "round", "tick", "worker",  "order", "task"]]
    action_df_12 = pd.concat([action_df_12, df_round])

action_df_12

  0%|          | 0/36 [00:00<?, ?it/s]


JSONDecodeError: Expecting ':' delimiter: line 1 column 6 (char 5)

In [61]:
json.loads(df["round_1_actions"][0])

JSONDecodeError: Expecting ':' delimiter: line 1 column 6 (char 5)

In [ ]:

ooo = action_df_12.loc[action_df_12["round"] == "1", :]
bbb = ooo[ooo["ResponseId"] ==  "R_sKYZSwmkYqsCiZj"]
bbb[bbb["order"]  == 1]

,ResponseId,intervention,round,tick,worker,order,task


for round 3-6

In [ ]:
# ticks is array; json.loads(...)
def player_round36(ticks):
  tick_array = []
  time = 1
  worker_this_round = []
  order_this_round = []
  task_this_round = []
  action_this_tick = []
  for tick in ticks:
    inn = False
    worker_this_tick = {1:1, 2:2} #key is worker
    order_this_tick = {1:6, 2:6} #key is worker
    task_this_tick = {1:6, 2:6} #key is worker
    for action in tick:
      if list(action.items()) not in action_this_tick:
        worker = action["worker_index"] + 1
        worker_this_tick[worker] = worker
        order_this_tick[worker] = action["order_index"]
        task_this_tick[worker] = action["task"]
        action_this_tick.append(list(action.items()))        
    time_this_tick = [time for _ in np.arange(2)]
    tick_array.extend(time_this_tick)
    time += 1
    worker_this_round.extend(list(worker_this_tick.values()))
    order_this_round.extend(list(order_this_tick.values()))
    task_this_round.extend(list(task_this_tick.values()))
  tick_array, worker_this_round, order_this_round, task_this_round = np.array(tick_array), np.array(worker_this_round), np.array(order_this_round), np.array(task_this_round)
  #print(tick_array)
  out = pd.DataFrame()
  out["tick"], out["worker"], out["order"], out["task"] = tick_array, worker_this_round, order_this_round, task_this_round
  return out

#player_round(json.loads(df["round_1_actions"][0]))

action_df_36 = pd.DataFrame()
for id in np.arange(len(df["ResponseId"])):
  for round in ["round_3_actions", "round_4_actions", "round_5_actions", "round_6_actions"]:
    df_round = player_round36(json.loads(df[round][id]))
    length = len(df_round.iloc[:,1])
    ResponseId = [df.iloc[id, 0] for _ in np.arange(length)]
    intervention = [df.iloc[id, 1] for _ in np.arange(length)]
    round_array = [round.split("_")[1] for _ in np.arange(length)]
    df_round["ResponseId"] = ResponseId
    df_round["intervention"] = intervention
    df_round["round"] = round_array
    df_round = df_round[["ResponseId", "intervention", "round", "tick", "worker",  "order", "task"]]
    action_df_36 = pd.concat([action_df_36, df_round])

action_df_36

,ResponseId,intervention,round,tick,worker,order,task
0,R_ZIZguiYOWRX744x,paysocial,3,1,1,0,0
1,R_ZIZguiYOWRX744x,paysocial,3,1,2,1,0
2,R_ZIZguiYOWRX744x,paysocial,3,2,1,6,6
3,R_ZIZguiYOWRX744x,paysocial,3,2,2,6,6
4,R_ZIZguiYOWRX744x,paysocial,3,3,1,6,6
...,...,...,...,...,...,...,...
61,R_oXk1uY70EL4SaM9,social,6,31,2,6,6
62,R_oXk1uY70EL4SaM9,social,6,32,1,6,6
63,R_oXk1uY70EL4SaM9,social,6,32,2,6,6
64,R_oXk1uY70EL4SaM9,social,6,33,1,6,6


concat 1-6

In [ ]:
action_df = pd.concat([action_df_12, action_df_36])
action_df

,ResponseId,intervention,round,tick,worker,order,task
0,R_ZIZguiYOWRX744x,paysocial,1,1,0,0,0
1,R_ZIZguiYOWRX744x,paysocial,1,1,1,1,0
2,R_ZIZguiYOWRX744x,paysocial,1,1,2,2,0
3,R_ZIZguiYOWRX744x,paysocial,1,2,1,6,6
4,R_ZIZguiYOWRX744x,paysocial,1,2,1,6,6
...,...,...,...,...,...,...,...
61,R_oXk1uY70EL4SaM9,social,6,31,2,6,6
62,R_oXk1uY70EL4SaM9,social,6,32,1,6,6
63,R_oXk1uY70EL4SaM9,social,6,32,2,6,6
64,R_oXk1uY70EL4SaM9,social,6,33,1,6,6


In [ ]:
action_df.to_csv("action_data_frame.csv")

In [ ]:
tem = action_df.loc[action_df.task != 6,:]
pivoting = pd.pivot_table(tem, values="order", index=["ResponseId", "round", "worker"],
                    columns=["task"], aggfunc=len)
# pivoting = pivoting.rename(columns = {"intervention":  "total assignment frequency"})
unstack = pivoting.unstack("worker")
final_action_df = tem.merge(unstack, how = "left", left_on = ["ResponseId", "round"], right_index=True)
final_action_df = final_action_df.rename(columns = 
                                         {(0, 0): "chef_round_chopping", (1, 0): "chef_round_cooking", (2, 0): "chef_round_plating",
                                          (0, 1): "souschef_round_chopping", (1, 1): "souschef_round_cooking", (2, 1): "souschef_round_plating",
                                          (0, 2): "server_round_chopping", (1, 2): "server_round_cooking", (2, 2): "server_round_plating",
                                          })
final_action_df = final_action_df.fillna(0)
final_action_df

/Users/mingyuanma/opt/anaconda3/lib/python3.8/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


,ResponseId,intervention,round,tick,worker,order,task,chef_round_chopping,souschef_round_chopping,server_round_chopping,chef_round_cooking,souschef_round_cooking,server_round_cooking,chef_round_plating,souschef_round_plating,server_round_plating
0,R_ZIZguiYOWRX744x,paysocial,1,1,0,0,0,1.0,1.0,2.0,3.0,1.0,0.0,0.0,2.0,2.0
1,R_ZIZguiYOWRX744x,paysocial,1,1,1,1,0,1.0,1.0,2.0,3.0,1.0,0.0,0.0,2.0,2.0
2,R_ZIZguiYOWRX744x,paysocial,1,1,2,2,0,1.0,1.0,2.0,3.0,1.0,0.0,0.0,2.0,2.0
6,R_ZIZguiYOWRX744x,paysocial,1,3,0,0,1,1.0,1.0,2.0,3.0,1.0,0.0,0.0,2.0,2.0
16,R_ZIZguiYOWRX744x,paysocial,1,6,1,1,1,1.0,1.0,2.0,3.0,1.0,0.0,0.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,R_oXk1uY70EL4SaM9,social,6,19,1,3,0,0.0,3.0,1.0,0.0,2.0,2.0,0.0,2.0,2.0
39,R_oXk1uY70EL4SaM9,social,6,20,2,2,1,0.0,3.0,1.0,0.0,2.0,2.0,0.0,2.0,2.0
42,R_oXk1uY70EL4SaM9,social,6,22,1,3,1,0.0,3.0,1.0,0.0,2.0,2.0,0.0,2.0,2.0
60,R_oXk1uY70EL4SaM9,social,6,31,1,3,2,0.0,3.0,1.0,0.0,2.0,2.0,0.0,2.0,2.0


In [ ]:
final_action_df.to_csv("final_data_frame.csv")

### did not follow the tip with frequency more than or equal to 3

In [ ]:
with3 = final_action_df.loc[(final_action_df["server_round_cooking"] >= 3)&(final_action_df["round"].astype(int) >= 3),:]
with3

,ResponseId,intervention,round,tick,worker,order,task,chef_round_chopping,souschef_round_chopping,server_round_chopping,chef_round_cooking,souschef_round_cooking,server_round_cooking,chef_round_plating,souschef_round_plating,server_round_plating
0,R_2YfGUfsykAuyUpu,social,4,1,1,2,0,0.0,2.0,2.0,0.0,2.0,3.0,0.0,2.0,3.0
1,R_2YfGUfsykAuyUpu,social,4,1,2,1,0,0.0,2.0,2.0,0.0,2.0,3.0,0.0,2.0,3.0
2,R_2YfGUfsykAuyUpu,social,4,2,1,0,0,0.0,2.0,2.0,0.0,2.0,3.0,0.0,2.0,3.0
7,R_2YfGUfsykAuyUpu,social,4,4,2,2,1,0.0,2.0,2.0,0.0,2.0,3.0,0.0,2.0,3.0
8,R_2YfGUfsykAuyUpu,social,4,5,1,1,1,0.0,2.0,2.0,0.0,2.0,3.0,0.0,2.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36,R_3KU4TYOj74Q2Th3,pay,6,19,1,1,2,0.0,2.0,3.0,0.0,1.0,3.0,0.0,3.0,1.0
37,R_3KU4TYOj74Q2Th3,pay,6,19,2,3,0,0.0,2.0,3.0,0.0,1.0,3.0,0.0,3.0,1.0
60,R_3KU4TYOj74Q2Th3,pay,6,31,1,2,2,0.0,2.0,3.0,0.0,1.0,3.0,0.0,3.0,1.0
69,R_3KU4TYOj74Q2Th3,pay,6,35,2,3,1,0.0,2.0,3.0,0.0,1.0,3.0,0.0,3.0,1.0


In [ ]:
with3.to_csv("server_cooking_3.csv")

### Checking edge cases

In [ ]:
check = final_action_df.copy()
check["chopping"] = check.chef_round_chopping	+ check.souschef_round_chopping + check.server_round_chopping
check["cooking"] = check.chef_round_cooking	+ check.souschef_round_cooking + check.server_round_cooking
check["plating"] = check.chef_round_plating	+ check.souschef_round_plating + check.server_round_plating
# or check.cooking!=4 or check.plating!=4
condition = (check.chopping!=4)|(check.cooking!=4)|(check.plating!=4)
wrong = check.loc[condition, :]
wrong

,ResponseId,intervention,round,tick,worker,order,task,chef_round_chopping,souschef_round_chopping,server_round_chopping,chef_round_cooking,souschef_round_cooking,server_round_cooking,chef_round_plating,souschef_round_plating,server_round_plating,chopping,cooking,plating
0,R_yWxhQlM2cbKDdND,curriculum,1,1,0,0,0,3.0,2.0,0.0,3.0,1.0,0.0,0.0,3.0,2.0,5.0,4.0,5.0
1,R_yWxhQlM2cbKDdND,curriculum,1,1,1,1,0,3.0,2.0,0.0,3.0,1.0,0.0,0.0,3.0,2.0,5.0,4.0,5.0
3,R_yWxhQlM2cbKDdND,curriculum,1,2,0,2,0,3.0,2.0,0.0,3.0,1.0,0.0,0.0,3.0,2.0,5.0,4.0,5.0
6,R_yWxhQlM2cbKDdND,curriculum,1,3,0,0,1,3.0,2.0,0.0,3.0,1.0,0.0,0.0,3.0,2.0,5.0,4.0,5.0
7,R_yWxhQlM2cbKDdND,curriculum,1,3,1,2,0,3.0,2.0,0.0,3.0,1.0,0.0,0.0,3.0,2.0,5.0,4.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,R_1dzyR4tOshqPykF,curriculum,6,20,2,1,2,0.0,3.0,3.0,0.0,3.0,1.0,0.0,2.0,4.0,6.0,4.0,6.0
43,R_1dzyR4tOshqPykF,curriculum,6,22,2,3,0,0.0,3.0,3.0,0.0,3.0,1.0,0.0,2.0,4.0,6.0,4.0,6.0
49,R_1dzyR4tOshqPykF,curriculum,6,25,2,2,2,0.0,3.0,3.0,0.0,3.0,1.0,0.0,2.0,4.0,6.0,4.0,6.0
50,R_1dzyR4tOshqPykF,curriculum,6,26,1,3,1,0.0,3.0,3.0,0.0,3.0,1.0,0.0,2.0,4.0,6.0,4.0,6.0


In [ ]:
wrong2 = wrong.groupby(["ResponseId", "round"]).count().iloc[:,[0]].rename(columns = {"intervention":"times"})
len(np.unique(np.array([a[0] for a in list(wrong2.index)])))

520

In [ ]:
# &(action_df["order"] == 6)&(action_df["task"] == 6)&(action_df["worker"] ==2)

In [ ]:
one = action_df[(action_df.ResponseId == "R_12g85TXz0g5zQSv")&(action_df["round"] == "2")]
two = one[one["task"] != 6]
two

,ResponseId,intervention,round,tick,worker,order,task


In [ ]:

json.loads(df[df.ResponseId == "R_12g85TXz0g5zQSv"]["round_2_actions"][119])

KeyError: 119

In [ ]:
df[df.ResponseId == "R_12g85TXz0g5zQSv"]["round_2_actions"]

### check edge cases

In [ ]:
final_action_df.to_csv("action_data_frame.csv")

### Start on state